## Singular Value Descomposition (SVD) Matrix Factorization

In [1]:
import pandas as pd
import numpy as np

from sklearn.decomposition import TruncatedSVD

The MovieLens dataset by the GroupLens Research Project at the University of Minnesota from https://grouplens.org/datasets/movielens/100k/

In [2]:
data = pd.read_csv('u.data', sep='\t', names=['user_id','item_id', 'rating','timestamp'])
data.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [3]:
columns = ['item_id', 'movie title', 'release date', 'video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
          'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
          'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

movies = pd.read_csv('u.item', sep='|', names=columns, encoding='latin-1')
movie_names = movies[['item_id', 'movie title']]
movie_names.head()

,item_id,movie title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


We combine the two data frames

In [4]:
combined_movies = pd.merge(data, movie_names, on="item_id")
combined_movies.head()

,user_id,item_id,rating,timestamp,movie title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


More than one person reviewed the same movie. To see which movie got the most reviews, we group the data by item_id and count the ratings

In [9]:
combined_movies.groupby('item_id')['rating'].count().sort_values(ascending=False).head()

item_id
50     583
258    509
100    508
181    507
294    485
Name: rating, dtype: int64

In [12]:
filter = combined_movies['item_id']==50
combined_movies[filter]['movie title'].unique()

array(['Star Wars (1977)'], dtype=object)

## Utility matrix

This matrix has a value for each reviewer and movie

In [13]:
rating_crosstab = combined_movies.pivot_table(values = 'rating', index='user_id', columns='movie title', fill_value=0)
rating_crosstab.head()

movie title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,2,5,0,0,3,4,0,0,...,0,0,0,5,3,0,0,0,4,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,2,0,0,0,0,4,0,0,...,0,0,0,4,0,0,0,0,4,0


We will now transpose this and use SVD in order to decompose it in synthetic representions of the user reviews

In [15]:
rating_crosstab.shape

(943, 1664)

In [16]:
x = rating_crosstab.values.T
x.shape

(1664, 943)

#### SVD

In [18]:
SVD = TruncatedSVD(n_components=12, random_state=17)
result = SVD.fit_transform(x)
result.shape

(1664, 12)

We calculate the Pearson r correlation coefficient for the result matrix

In [19]:
correlation = np.corrcoef(result)
correlation.shape

(1664, 1664)

Let's isolate the movie that has the most reviews item_id = 50, Star Wars (1977)

In [23]:
movie_names = rating_crosstab.columns
list_movies = list(movie_names)
starwars = list_movies.index('Star Wars (1977)')
starwars

1398

In [26]:
corr_starwars = correlation[starwars]
corr_starwars.shape

(1664,)

corr_starwars contains a Pearson r coefficient telling us how much each of the 1664 movies correlate with Star Wars based on user preferences

#### Movies with high correlation to Star Wars (1977)

In [28]:
list(movie_names[(corr_starwars<1.0) & (corr_starwars>0.9)])

['Die Hard (1988)',
 'Empire Strikes Back, The (1980)',
 'Fugitive, The (1993)',
 'Raiders of the Lost Ark (1981)',
 'Return of the Jedi (1983)',
 'Terminator 2: Judgment Day (1991)',
 'Terminator, The (1984)',
 'Toy Story (1995)']

### Top correlated movie

In [31]:
list(movie_names[(corr_starwars<1.0) & (corr_starwars>0.95)])

['Return of the Jedi (1983)']